# Modeling GNN-ePC-SAFT

## Starting point

In [ ]:
#    import os
#    import torch
#    os.environ['TORCH'] = torch.__version__
#    print(torch.__version__)

#   !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
%pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
%pip install rdkit torchmetrics ml-collections polars clu contextlib2 -q --no-deps
!mkdir pcsaft
!cp -a /kaggle/input/pcsaftpkg/PC-SAFT-1.4.1/. /kaggle/working/pcsaft
%pip install ./pcsaft -q
%pip install ogb -q

!pip show pcsaft | grep -E "Version|Name"

!lscpu | grep -E "Model name|Core|core" 

!nvidia-smi

In [ ]:
import os, shutil

#os.environ["CUDA_VISIBLE_DEVICES"]="0"

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("ghtk")
secret_value_1 = user_secrets.get_secret("kaggle")
secret_value_2 = user_secrets.get_secret("username")
secret_value_3 = user_secrets.get_secret("wandbkey")
os.environ["KAGGLE_USERNAME"]=secret_value_2
os.environ["KAGGLE_KEY"]=secret_value_1
import wandb
os.environ['WANDB_API_KEY'] = secret_value_3
wandb.login()

In [ ]:
# for the cloud
!git clone https://{secret_value_0}@github.com/wildsonbbl/GNN-ePC-SAFT.git --branch san --single-branch -q
%cd GNN-ePC-SAFT/

!mkdir /kaggle/working/GNN-ePC-SAFT/train/checkpoints
!cp /kaggle/input/gnnepcsaft/* /kaggle/working/GNN-ePC-SAFT/train/checkpoints/

## Training

In [ ]:
# shutil.copy("./train/checkpoints/model3-5_00e6.pth", "./train/checkpoints/ra_last_checkpoint.pth")

In [ ]:
!python -m train.train --workdir=./ --config=configs/default.py --dataset=ramirez\
--config.num_train_steps=500_000 --config.checkpoint_every_steps=25_000 \
--config.eval_every_steps=25_000 --config.log_every_steps=5_000 \
--config.model=PNA --config.optimizer=adam --config.learning_rate=1e-3 \
--config.change_opt=False --config.change_sch=False \
--config.propagation_depth=2 --config.hidden_dim=128 --config.num_mlp_layers=1 --config.pre_layers=1 --config.post_layers=3 \
--config.add_self_loops=False --config.skip_connections=False

### For updating kaggle dataset

In [ ]:
!kaggle datasets metadata -p /kaggle/working/GNN-ePC-SAFT/train/checkpoints wildsonbblima/gnnepcsaft

shutil.copy("./train/checkpoints/ra_last_checkpoint.pth", "./train/checkpoints/model5-0_50e6.pth")

os.remove("./train/checkpoints/ra_last_checkpoint.pth")

!kaggle datasets version -p /kaggle/working/GNN-ePC-SAFT/train/checkpoints -m "model 5 at 0.50e6 steps"

print("finished")

### Clear workspace

In [ ]:
%cd ../
!rm -rf *